<a href="https://colab.research.google.com/github/Ganeshs170220/AI-MODELS/blob/main/AI_GENERATE_QUESTION_AND_ANSWERS_FROM_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AI GENERATE QUESTION AND ANSWERS FROM PDF

#Installing the required modules

In [ ]:
!pip install PyPDF2
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.1 MB/s eta 0:00:00


#Installing the model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")
model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")


#Read the pdf and convert into text


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import PyPDF2

def convert_pdf_to_text(pdf_path):
    # Code to convert PDF to text using PyPDF2
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

# Provide the path to your PDF file
pdf_path = '/content/drive/MyDrive/Colab Notebooks/pdf/example.pdf'
PDF_TEXT = convert_pdf_to_text(pdf_path)

#printing the pdf text

In [6]:
PDF_TEXT

'Chapter 1 Preview\n 1\nArtificial Intelligence\nIndex Report 2023\nArtificial Intelligence\nIndex Report 2023\nCHAPTER 1:  \nResearch and \nDevelopmentChapter 1 Preview\n 2\nArtificial Intelligence\nIndex Report 2023Overview   3\nChapter Highlights   4\n1.1 Publications    5\nOverview   5\n Total Number of AI Publications   5\n By Type of Publication   6\n By Field of Study   7\n By Sector   8\n Cross-Country Collaboration  10\n Cross-Sector Collaboration  12\nAI Journal Publications  13\n Overview  13\n By Region  14\n By Geographic Area  15\n Citations  16\nAI Conference Publications  17\n Overview  17\n By Region  18\n By Geographic Area  19\n Citations 20\nAI Repositories 21\n Overview 21\n By Region 22\n By Geographic Area 23\n Citations 24\n Narrative Highlight:   \nTop Publishing Institutions 25\n All Fields 25 Computer Vision 27\n Natural Language Processing 28\n Speech Recognition 29\n1.2 Trends in Significant  \nMachine Learning Systems 30\nGeneral Machine Learning Systems 3

#Data cleaning

In [7]:
PDF_TEXT = PDF_TEXT.replace('.', '.<eos>')
PDF_TEXT = PDF_TEXT.replace('?', '?<eos>')
PDF_TEXT = PDF_TEXT.replace('!', '!<eos>')
PDF_TEXT = PDF_TEXT.replace('\n','')

#After data cleaning

In [8]:
PDF_TEXT

'Chapter 1 Preview 1Artificial IntelligenceIndex Report 2023Artificial IntelligenceIndex Report 2023CHAPTER 1:  Research and DevelopmentChapter 1 Preview 2Artificial IntelligenceIndex Report 2023Overview   3Chapter Highlights   41.<eos>1 Publications    5Overview   5 Total Number of AI Publications   5 By Type of Publication   6 By Field of Study   7 By Sector   8 Cross-Country Collaboration  10 Cross-Sector Collaboration  12AI Journal Publications  13 Overview  13 By Region  14 By Geographic Area  15 Citations  16AI Conference Publications  17 Overview  17 By Region  18 By Geographic Area  19 Citations 20AI Repositories 21 Overview 21 By Region 22 By Geographic Area 23 Citations 24 Narrative Highlight:   Top Publishing Institutions 25 All Fields 25 Computer Vision 27 Natural Language Processing 28 Speech Recognition 291.<eos>2 Trends in Significant  Machine Learning Systems 30General Machine Learning Systems 30 System Types 30 Sector Analysis 31 National Affiliation 32  Systems 32  Au

#Creating chunks

In [9]:
max_chunk = 600
sentences = PDF_TEXT.split('<eos>')
current_chunk = 0
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


#Printing the length of the chunks

In [10]:
length = len(chunks)
length

19

#AI Generating the questions and answers

In [11]:
count = 1
for i in range(0,length):
  inputs = tokenizer(chunks[i], return_tensors="pt")
  outputs = model.generate(**inputs, max_length=100)
  question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
  question_answer = question_answer.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
  question, answer = question_answer.split(tokenizer.sep_token)
  print(f'question-{count}: {question}')
  print("answer:", answer)
  print("----------------------")
  count +=1

Token indices sequence length is longer than the specified maximum sequence length for this model (799 > 512). Running this sequence through the model will result in indexing errors


question-1:  What is the largest number of cross country collaborations between the United States and China?
answer:  4 times
----------------------
question-2:  What was the estimated cost of training PaLM?
answer:  $50,000 USD
----------------------
question-3:  What is the largest number of cross-country collaborations between the United States and China?
answer:  2. 5 times
----------------------
question-4:  What is the percentage of AI journal publications in the United States?
answer:  15. 03%
----------------------
question-5:  What was the total number of AI conference publications in 2021?
answer:  85,094
----------------------
question-6:  What is the percentage of AI conference publications in the world?
answer:  26. 2%
----------------------
question-7:  What is the name of the top publishing institution in China?
answer:  Tsinghua University
----------------------
question-8:  What was the most common system type in 2022?
answer:  language
----------------------
question-